# <span style="color:rgb(255, 0, 255)">  Lab | Feature Engineering</span> 
### <span style="color:rgb(255, 0, 255)">Ainara Guerra</span> 

### <span style="color:rgb(255, 0, 255)">--- Import the necessary libraries</span>

In [1]:
import pandas as pd
import numpy as np
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import matplotlib.ticker as mk
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

import os #we will use the function listdir to list files in a folder if needed
import math #to apply absolute value if needed

### <span style="color:rgb(255, 0, 255)">--- Load the database</span>

In [2]:
data = pd.read_csv('C:/Users/narag/OneDrive/Documentos/Ironhack/Week 16/7.01/learningSet.csv', low_memory=False)

### <span style="color:rgb(255, 0, 255)">--- Let's look at its main features (head, shape, info).

In [3]:
data.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,,,,XXXX,T2,36,60.0,,,,,,,NaN,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,9706,9606.0,9604.0,9604.

In [4]:
data.shape


(95412, 481)

***

### Previous labs starts here:

### <span style="color:rgb(255, 0, 255)"> --- Check for null values in all the columns

In [5]:
null_values_per_column = pd.DataFrame(data.isnull().sum(), columns=['Null Count'])
null_values_per_column.index.name = 'Column'
null_values_per_column = null_values_per_column.sort_values('Null Count', ascending=False)
null_values_per_column

,Null Count
Column,
RDATE_5,95403
RAMNT_5,95403
RDATE_3,95170
RAMNT_3,95170
RDATE_4,95131
...,...
ETHC3,0
ETHC2,0
ETHC1,0


### <span style="color:rgb(255, 0, 255)"> --- Identify columns that over 85% missing values and remove them



In [6]:
# First let's sort these columns to check which are the ones with more than 85% nulls as a dataframe
nulls = pd.DataFrame(data.isna().sum() * 100 / len(data), columns=['percentage'])
nulls_sorted = nulls[nulls['percentage'] >= 85].sort_values('percentage', ascending=False)
nulls_sorted

,percentage
RDATE_5,99.990567
RAMNT_5,99.990567
RDATE_3,99.746363
RAMNT_3,99.746363
RDATE_4,99.705488
RAMNT_4,99.705488
RDATE_6,99.186685
RAMNT_6,99.186685
RAMNT_15,92.388798
RDATE_15,92.388798


In [7]:
# Now that we already check it, let's create a list so we can remove them.
missing_percentages = data.isnull().mean() * 100
columns_to_drop = missing_percentages[missing_percentages > 85].index.tolist()
data.drop(columns_to_drop, axis=1, inplace=True)
data.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_8,RDATE_9,RDATE_11,RDATE_12,RDATE_14,RDATE_16,RDATE_18,RDATE_19,RDATE_22,RDATE_24,RAMNT_8,RAMNT_9,RAMNT_11,RAMNT_12,RAMNT_14,RAMNT_16,RAMNT_18,RAMNT_19,RAMNT_22,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,,,,XXXX,T2,36,60.0,,,,,,,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,9706,9606.0,9604.0,9604.0,9603.0,9602.0,9601.0,9511.0,9510.0,9510.0,9508.0,9507.0,9506.0,9504.0,9503.0,9502.0,9501.0,9411.0,9411.0,9410.0,9409.0,9407.0,9406.0,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E

### <span style="color:rgb(255, 0, 255)"> --- Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

OSOURCE - symbol definitions not provided, too many categories
    
ZIP CODE - we are including state already



In [8]:
drop_list = []

drop_list.append('OSOURCE')
drop_list.append('ZIP')

data.drop(drop_list, axis=1, inplace=True)
data.head()

,ODATEDW,TCODE,STATE,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_8,RDATE_9,RDATE_11,RDATE_12,RDATE_14,RDATE_16,RDATE_18,RDATE_19,RDATE_22,RDATE_24,RAMNT_8,RAMNT_9,RAMNT_11,RAMNT_12,RAMNT_14,RAMNT_16,RAMNT_18,RAMNT_19,RAMNT_22,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,0,IL,,,3712,0,,,,,XXXX,T2,36,60.0,,,,,,,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,9706,9606.0,9604.0,9604.0,9603.0,9602.0,9601.0,9511.0,9510.0,9510.0,9508.0,9507.0,9506.0,9504.0,9503.0,9502.0,9501.0,9411.0,9411.0,9410.0,9409.0,9407.0,9406.0,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,27,9702,74,6,

### <span style="color:rgb(255, 0, 255)"> --- Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest

 <span style="color:rgb(255, 0, 255)"> Note that there are a few null values in the column. We will first replace those null values using the code below:
    
        print(categorical['GENDER'].value_counts())
        categorical['GENDER'] = categorical['GENDER'].fillna('F')

In [9]:
data["GENDER"].value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [10]:
data['GENDER'] = data['GENDER'].fillna('F')

In [11]:
data["GENDER"].value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [12]:
# the code given in the lab doesn't work, let's try another thing:
data['GENDER'] = data['GENDER'].replace(" ", "F")

In [13]:
data["GENDER"].value_counts()
#Now It worked. 

F    54234
M    39094
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [14]:
data['GENDER'] = data['GENDER'].replace(['U', 'J', 'C', 'A'], 'other')

In [15]:
data["GENDER"].value_counts()

F        54234
M        39094
other     2084
Name: GENDER, dtype: int64

***

### Activities 7.02 & 7.03

7.01.01 Remove  MAILCODE, NOEXCH, and MDMAUD because of imbalance data

<!-- The activity 7.01.01 told us to delete: MAILCODE, NOEXCH & MDMAUD -->

In [16]:
data["MAILCODE"].value_counts()

     94013
B     1399
Name: MAILCODE, dtype: int64

In [17]:
data["NOEXCH"].value_counts()

0    95085
1      285
X       35
         7
Name: NOEXCH, dtype: int64

In [18]:
data["MDMAUD"].value_counts()

XXXX    95118
C1CM       65
L1CM       44
I1CM       37
D2CM       28
C2CM       24
D1CM       20
L2CM       15
L1LM        8
C1LM        8
I2CM        7
D5CM        5
D5MM        5
D2MM        4
C5CM        3
C2MM        3
C2LM        3
D5TM        3
I1LM        3
I5CM        1
C1MM        1
I5MM        1
C5MM        1
I2MM        1
L1MM        1
L2LM        1
C5TM        1
L2TM        1
Name: MDMAUD, dtype: int64

In [19]:
drop_list = []

drop_list.append('MAILCODE')
drop_list.append('NOEXCH')
drop_list.append('MDMAUD')
data.drop(drop_list, axis=1, inplace=True)
data.head()

,ODATEDW,TCODE,STATE,PVASTATE,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_8,RDATE_9,RDATE_11,RDATE_12,RDATE_14,RDATE_16,RDATE_18,RDATE_19,RDATE_22,RDATE_24,RAMNT_8,RAMNT_9,RAMNT_11,RAMNT_12,RAMNT_14,RAMNT_16,RAMNT_18,RAMNT_19,RAMNT_22,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,0,IL,,3712,,,,,T2,36,60.0,,,,,,,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,9706,9606.0,9604.0,9604.0,9603.0,9602.0,9601.0,9511.0,9510.0,9510.0,9508.0,9507.0,9506.0,9504.0,9503.0,9502.0,9501.0,9411.0,9411.0,9410.0,9409.0,9407.0,9406.0,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,27,9702,74,6,14,NaN,NaN,NaN,NaN,9507.0,9505.

7.02.01 replace null values in the columns DATASRCE and GEOCODE2.

In [20]:
data["DATASRCE"].value_counts()
#we told on class that the best way to arrange this was to put a 0 on " " rows.

3    43549
2    23455
     21280
1     7128
Name: DATASRCE, dtype: int64

In [21]:
data['DATASRCE'] = data['DATASRCE'].replace(" ", "0")

In [22]:
data["DATASRCE"].value_counts()

3    43549
2    23455
0    21280
1     7128
Name: DATASRCE, dtype: int64

In [23]:
data["GEOCODE2"].value_counts()
# In this case, as they are very little values, I think it is better to drop them

A    34484
B    28505
D    16580
C    15524
       187
Name: GEOCODE2, dtype: int64

In [24]:
data.drop(data[data["GEOCODE2"] == " "].index, inplace=True)

In [25]:
data["GEOCODE2"].value_counts()

A    34484
B    28505
D    16580
C    15524
Name: GEOCODE2, dtype: int64

7.02.02 Remove the columns starting with ADATE_. We are assuming that the date when the previous mail was done is not significant in the respondents decision to give donation

In [26]:
columns_to_remove = [col for col in data.columns if col.startswith("ADATE_")]
columns_to_remove

['ADATE_2',
 'ADATE_3',
 'ADATE_4',
 'ADATE_5',
 'ADATE_6',
 'ADATE_7',
 'ADATE_8',
 'ADATE_9',
 'ADATE_10',
 'ADATE_11',
 'ADATE_12',
 'ADATE_13',
 'ADATE_14',
 'ADATE_15',
 'ADATE_16',
 'ADATE_17',
 'ADATE_18',
 'ADATE_19',
 'ADATE_20',
 'ADATE_21',
 'ADATE_22',
 'ADATE_23',
 'ADATE_24']

In [27]:
data["ADATE_2"].value_counts()
#they are only 13 columns with the value 9704

9706    95212
9704       13
Name: ADATE_2, dtype: int64

In [28]:
data = data.drop(columns=columns_to_remove)

### Current lab starts here:

In [29]:
# We start with this dataset

data.head()

,ODATEDW,TCODE,STATE,PVASTATE,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_8,RDATE_9,RDATE_11,RDATE_12,RDATE_14,RDATE_16,RDATE_18,RDATE_19,RDATE_22,RDATE_24,RAMNT_8,RAMNT_9,RAMNT_11,RAMNT_12,RAMNT_14,RAMNT_16,RAMNT_18,RAMNT_19,RAMNT_22,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,0,IL,,3712,,,,,T2,36,60.0,,,,,,,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,27,9702,74,6,14,NaN,NaN,NaN,NaN,9507.0,9505.0,NaN,NaN,NaN,9406.0,NaN,NaN,NaN,NaN,10.0,11.0,NaN,NaN,NaN,9.0,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,9003.0,4.0,7.741935,95515,0,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,1,CA,,5202,,,,,S1,14,46.0,E,H,,,,M,6.0,M,9.0,16,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,3,0,15,55,11,6,2,1,,,,9.0,2,,,,,,,,,,,,,,,,,,,,,3611,940,998,99,0,0,50,50,67,0,0,31,

### <span style="color:rgb(255, 0, 255)"> --- Here we will work on cleaning some of the other columns in the dataset using the techniques that we used before in the lessons.

<span style="color:rgb(255, 0, 255)"> Check for null values in the numerical columns.

In [30]:
# trying to find patterns in the null values: Columns that start with RAMNT and RDATE have a lot of missing values still:
columns_to_check = data.columns[data.columns.str.startswith(('RAMNT', 'RDATE'))]
null_values_filtered = pd.DataFrame(data[columns_to_check].isnull().sum(), columns=['Null Count'])
null_values_filtered.index.name = 'Column'
null_values_filtered = null_values_filtered.sort_values('Null Count', ascending=False)
null_values_filtered

,Null Count
Column,
RDATE_11,80516
RAMNT_11,80516
RAMNT_19,79373
RDATE_19,79373
RDATE_9,78512
RAMNT_9,78512
RAMNT_24,77516
RDATE_24,77516
RAMNT_18,75489


In [31]:
# Let's dive into RAMNTALL:
data["RAMNTALL"].value_counts()

20.00      4195
25.00      3685
15.00      3389
30.00      2569
40.00      1589
           ... 
26.45         1
305.07        1
126.95        1
148.03        1
3775.00       1
Name: RAMNTALL, Length: 2093, dtype: int64

In [32]:
# Look's good to me so we are going to remove all these RAMNTALL:
null_values_filtered = null_values_filtered[null_values_filtered.index != 'RAMDTALL']

In [33]:
columns_to_drop_2 = [col for col in data.columns if col.startswith('RAMNT') or col.startswith('RDATE') and col != 'RAMDTALL']
data = data.drop(columns=columns_to_drop_2)

In [34]:
data.head()

,ODATEDW,TCODE,STATE,PVASTATE,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,0,IL,,3712,,,,,T2,36,60.0,,,,,,,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,27,9702,74,6,14,31,14,5.0,9208,12.0,9402,10.0,9512,8911,9003.0,4.0,7.741935,95515,0,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,1,CA,,5202,,,,,S1,14,46.0,E,H,,,,M,6.0,M,9.0,16,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,3,0,15,55,11,6,2,1,,,,9.0,2,,,,,,,,,,,,,,,,,,,,,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4

In [35]:
# let's review again the situation with nulls: 
null_values_per_column = pd.DataFrame(data.isnull().sum(), columns=['Null Count'])
null_values_per_column.index.name = 'Column'
null_values_per_column = null_values_per_column.sort_values('Null Count', ascending=False)
null_values_per_column_highest = null_values_per_column.head(25)
null_values_per_column_highest

,Null Count
Column,
MBCOLECT,52745
PUBOPP,52685
PUBCULIN,52685
MBBOOKS,52685
MAGFAML,52685
MAGFEM,52685
MAGMALE,52685
PUBGARDN,52685
MBGARDEN,52685


In [36]:
#there is still some columns that have more that 50% of their data missing, let's drop them
threshold = len(data) * 0.5  # Set the threshold to 50% of the total rows
columns_to_drop = data.columns[data.isnull().sum() > threshold]
data = data.drop(columns=columns_to_drop)

In [37]:
null_values_per_column = pd.DataFrame(data.isnull().sum(), columns=['Null Count'])
null_values_per_column.index.name = 'Column'
null_values_per_column = null_values_per_column.sort_values('Null Count', ascending=False)
null_values_per_column_highest = null_values_per_column.head(10)
null_values_per_column_highest

,Null Count
Column,
WEALTH1,44575
WEALTH2,43682
AGE,23540
INCOME,21179
NEXTDATE,9946
TIMELAG,9946
ADI,132
DMA,132
GEOCODE2,132


In [38]:
# I am going to create a function for doing this steps because I will do them repeatly in this notebook.
ef describe_column(dataset, column_name):
    column = dataset[column_name]
    unique_values = column.unique()
    value_counts = column.value_counts()
    null_count = column.isna().sum()
    
    print("Unique Values:")
    print(unique_values)
    print("\nValue Counts:")
    print(value_counts)
    print("\nNull Count:")
    print(null_count)

In [39]:
# let's dive a little bit more into the nulls in these columns: WEALTH1, WEALTH2, AGE, INCOME, TIMELAG, NEXTDATE

**Wealth 2**

In [40]:
describe_column(data, "WEALTH2")
# after reviewing the description of the data, I decided to drop this ond because is very similar to WEALTH1, to which we will be doing data engineering.

Unique Values:
[ 5.  9.  1.  0. nan  3.  2.  6.  8.  4.  7.]

Value Counts:
9.0    6517
8.0    5973
7.0    5682
6.0    5489
5.0    5348
4.0    5067
3.0    5042
2.0    4966
1.0    4189
0.0    3270
Name: WEALTH2, dtype: int64

Null Count:
43682


In [41]:
data = data.drop(columns=["WEALTH2"])

**Timelag**

In [42]:
describe_column(data, "TIMELAG")

Unique Values:
[4.000e+00 1.800e+01 1.200e+01 9.000e+00 1.400e+01 6.000e+00 8.000e+00
 7.000e+00       nan 3.000e+00 5.000e+00 1.000e+00 2.300e+01 1.700e+01
 2.000e+00 1.600e+01 1.500e+01 1.100e+01 1.000e+01 2.000e+01 0.000e+00
 2.100e+01 1.300e+01 1.900e+01 2.500e+01 2.600e+01 2.800e+01 3.100e+01
 2.200e+01 3.500e+01 2.700e+01 3.400e+01 4.200e+01 2.900e+01 3.000e+01
 4.900e+01 2.400e+01 6.200e+01 3.700e+01 3.600e+01 4.400e+01 4.600e+01
 3.900e+01 4.500e+01 3.800e+01 3.300e+01 4.800e+01 4.700e+01 4.300e+01
 4.000e+01 9.000e+01 5.600e+01 3.200e+01 5.700e+01 5.800e+01 5.900e+01
 4.100e+01 5.000e+01 5.100e+01 5.500e+01 6.100e+01 5.400e+01 7.700e+01
 8.900e+01 1.044e+03 5.300e+01 4.420e+02 5.200e+01 1.088e+03]

Value Counts:
5.0       8569
4.0       8474
3.0       8472
6.0       6636
2.0       6261
          ... 
89.0         1
77.0         1
61.0         1
58.0         1
1088.0       1
Name: TIMELAG, Length: 68, dtype: int64

Null Count:
9946


In [43]:
# Let's drop it
data = data.drop(columns=["TIMELAG"])

**Nextdate**

In [44]:
describe_column(data, "NEXTDATE")

Unique Values:
[9003. 9504. 9101. 8711. 8005. 9407. 8705. 8809. 9404.   nan 9512. 8910.
 9211. 9306. 9509. 8906. 8707. 9507. 8801. 9312. 9303. 9301. 9304. 9112.
 9109. 9602. 9505. 9205. 9102. 8706. 8703. 9601. 9401. 9210. 8804. 9111.
 8904. 8902. 8710. 9411. 9409. 9110. 9503. 8704. 9209. 9001. 8803. 9405.
 9410. 9202. 9212. 9104. 9012. 9402. 8802. 8811. 9408. 8812. 9310. 9502.
 8612. 9510. 8405. 9511. 9207. 9506. 9508. 8708. 9412. 9204. 9309. 9406.
 8807. 8912. 9105. 9008. 9302. 9103. 9603. 9107. 9203. 9011. 9006. 9007.
 9106. 9501. 8702. 9206. 9004. 9311. 8907. 8806. 8709. 8909. 8911. 9010.
 9308. 9702. 9403. 9005. 8901. 8903. 8905. 9305. 9208. 8712. 9201. 9009.
 8408. 8810. 8808. 9610. 8805. 9307. 9002. 9108. 8701. 9608. 8611. 8908.
 9605. 8607. 7804. 8608. 8610. 8505. 9609. 8205. 8406. 7810. 8011. 8508.
 9607. 8512. 9604. 8609. 8309. 9612. 7211. 8307. 8602. 8407. 8511. 7711.
 7403. 8403. 8012. 8507. 8603. 9611. 8604. 8410. 9701. 8606. 8605. 8411.
 7705. 8401. 8510. 8107. 9606. 7408.

In [45]:
# Let's try dropping it

data = data.drop(columns=["NEXTDATE"])

**Age**

Here starts the good part. We have age, income and Wealth1 with a loooot of nulls. Inspired buy Supha's way of explaing her thoughts, here are mind: 
- I don't see the point of introducing them 0, because that could biased this columns, as well as filling them with the mean. 
- I chose to use KNN because: I can use the same code (I even tried to make a function, didn't work). I see both discrete and continous variables and this model worked for both. So in Age, Income and Wealth1 I applied KNN and then filled the NaNs with the predictited values. There are a lot of missing values and I don't know if this is still biasing the model. But I don't have the answer yet at this step. We will see in nexts steps. 

In [46]:
# See how it looks like
describe_column(data, "AGE")

Unique Values:
[60. 46. nan 70. 78. 38. 65. 75. 72. 44. 62. 82. 74. 54. 42. 84. 49. 69.
 88. 30. 51. 66. 61. 68. 83. 45. 40. 48. 34. 80. 36. 50. 41. 71. 47. 33.
 63. 64. 59. 77. 73. 87. 89. 86. 52. 76. 26. 58. 39. 92. 81. 56. 79. 37.
 67. 57. 85. 53. 55. 94. 93. 43. 97. 90. 31. 29. 91. 28. 32. 35. 21. 27.
 24. 96. 95. 20. 98. 19. 22. 25. 23. 13. 16. 18. 11.  2.  1.  9. 17.  7.
  6. 10.  4. 14.  8.  3. 15.]

Value Counts:
50.0    1929
76.0    1884
72.0    1811
68.0    1807
74.0    1801
        ... 
8.0        1
9.0        1
10.0       1
6.0        1
15.0       1
Name: AGE, Length: 96, dtype: int64

Null Count:
23540


In [47]:
# KNN prediction to missing values.
# Step 1: Split the dataset
data_train = data[data['AGE'].notnull()]
data_test = data[data['AGE'].isnull()]

# Step 2: Preprocess the data
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
X_train = data_train[numeric_columns].drop(['AGE', 'WEALTH1', 'INCOME'], axis=1)
y_train = data_train['AGE']

# Perform any necessary preprocessing steps on X_train
# Here, we'll use StandardScaler to standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 3: Impute missing values
imputer = KNNImputer()
X_train_scaled_imputed = imputer.fit_transform(X_train_scaled)

# Step 4: Train a KNN classifier
model = KNeighborsClassifier()
model.fit(X_train_scaled_imputed, y_train)

# Step 5: Predict missing values
X_test = data_test[numeric_columns].drop(['AGE', 'WEALTH1', 'INCOME'], axis=1)
X_test_scaled = scaler.transform(X_test)
X_test_scaled_imputed = imputer.transform(X_test_scaled)
predicted_age = model.predict(X_test_scaled_imputed)

# Step 6: Fill in the missing values
data_test['AGE'] = predicted_age

# Concatenate the filled data_test and data_train
data_filled = pd.concat([data_train, data_test])

# The "data_filled" dataframe now contains the original data with the missing values in the "AGE" column filled using KNN classifier


In [48]:
data.shape

(95225, 390)

In [49]:
data_filled.shape

(95225, 390)

In [50]:
# Filling NaNs with predicted values.
data["AGE"] = data_filled["AGE"]

In [51]:
data["AGE"].isna().sum()

0

**Wealth 1**

In [52]:
# Seeing how it looks like
describe_column(data, "WEALTH1")

Unique Values:
[nan  9.  1.  4.  2.  6.  0.  5.  8.  3.  7.]

Value Counts:
9.0    7583
8.0    6792
7.0    6194
6.0    5818
5.0    5275
4.0    4809
3.0    4234
2.0    4082
1.0    3452
0.0    2411
Name: WEALTH1, dtype: int64

Null Count:
44575


In [53]:
null_values_per_column = pd.DataFrame(data.isnull().sum(), columns=['Null Count'])
null_values_per_column.index.name = 'Column'
null_values_per_column = null_values_per_column.sort_values('Null Count', ascending=False)
null_values_per_column_highest = null_values_per_column.head(10)
null_values_per_column_highest

,Null Count
Column,
WEALTH1,44575
INCOME,21179
GEOCODE2,132
ADI,132
DMA,132
MSA,132
CLUSTER2,132
VC1,0
EC1,0


In [58]:
# KNN prediction to missing values.

# Step 1: Split the dataset
data_train = data[data['WEALTH1'].notnull()]
data_test = data[data['WEALTH1'].isnull()]

# Step 2: Preprocess the data
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
X_train = data_train[numeric_columns].drop(['WEALTH1', 'INCOME'], axis=1)
y_train = data_train['WEALTH1']

# Perform any necessary preprocessing steps on X_train
# Here, we'll use StandardScaler to standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 3: Impute missing values
imputer = KNNImputer()
X_train_scaled_imputed = imputer.fit_transform(X_train_scaled)

# Step 4: Train a KNN classifier
model = KNeighborsClassifier()
model.fit(X_train_scaled_imputed, y_train)

# Step 5: Predict missing values
X_test = data_test[numeric_columns].drop(['WEALTH1', 'INCOME'], axis=1)
X_test_scaled = scaler.transform(X_test)
X_test_scaled_imputed = imputer.transform(X_test_scaled)
predicted_wealth = model.predict(X_test_scaled_imputed)

# Step 6: Fill in the missing values
data_test['WEALTH1'] = predicted_wealth

# Concatenate the filled data_test and data_train
data_filled2 = pd.concat([data_train, data_test])

# The "data_filled" dataframe now contains the original data with the missing values in the "WEALTH1" column filled using KNN classifier


In [59]:
data.shape

(95225, 390)

In [61]:
data["WEALTH1"].isna().sum()

44575

In [62]:
data_filled2.shape

(95225, 390)

In [63]:
# Filling NaNs with predicted values.
data["WEALTH1"] = data_filled2["WEALTH1"]

In [64]:
data["WEALTH1"].isna().sum()

0

**Income**

In [65]:
#Seeing how it looks lik
describe_column(data, "INCOME")

Unique Values:
[nan  6.  3.  1.  4.  2.  7.  5.]

Value Counts:
5.0    15434
2.0    13101
4.0    12720
1.0     9015
3.0     8551
6.0     7766
7.0     7459
Name: INCOME, dtype: int64

Null Count:
21179


In [66]:
null_values_per_column = pd.DataFrame(data.isnull().sum(), columns=['Null Count'])
null_values_per_column.index.name = 'Column'
null_values_per_column = null_values_per_column.sort_values('Null Count', ascending=False)
null_values_per_column_highest = null_values_per_column.head(10)
null_values_per_column_highest

,Null Count
Column,
INCOME,21179
GEOCODE2,132
CLUSTER2,132
MSA,132
ADI,132
DMA,132
EIC14,0
OEDC5,0
OEDC4,0


In [67]:
# KNN prediction to missing values.
Step 1: Split the dataset
data_train = data[data['INCOME'].notnull()]
data_test = data[data['INCOME'].isnull()]

# Step 2: Preprocess the data
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
X_train = data_train[numeric_columns].drop(['INCOME'], axis=1)
y_train = data_train['INCOME']

# Perform any necessary preprocessing steps on X_train
# Here, we'll use StandardScaler to standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 3: Impute missing values
imputer = KNNImputer()
X_train_scaled_imputed = imputer.fit_transform(X_train_scaled)

# Step 4: Train a KNN classifier
model = KNeighborsClassifier()
model.fit(X_train_scaled_imputed, y_train)

# Step 5: Predict missing values
X_test = data_test[numeric_columns].drop(['INCOME'], axis=1)
X_test_scaled = scaler.transform(X_test)
X_test_scaled_imputed = imputer.transform(X_test_scaled)
predicted_income = model.predict(X_test_scaled_imputed)

# Step 6: Fill in the missing values
data_test['INCOME'] = predicted_income

# Concatenate the filled data_test and data_train
data_filled3 = pd.concat([data_train, data_test])

# The "data_filled" dataframe now contains the original data with the missing values in the "INCOME" column filled using KNN classifier


In [68]:
data.shape

(95225, 390)

In [69]:
data["INCOME"].isna().sum()

21179

In [70]:
data_filled3.shape

(95225, 390)

In [71]:
# Filling NaNs with predicted values.
data["INCOME"] = data_filled3["INCOME"]

In [72]:
data["INCOME"].isna().sum()

0

In [73]:
null_values_per_column = pd.DataFrame(data.isnull().sum(), columns=['Null Count'])
null_values_per_column.index.name = 'Column'
null_values_per_column = null_values_per_column.sort_values('Null Count', ascending=False)
null_values_per_column_highest = null_values_per_column.head(10)
null_values_per_column_highest

,Null Count
Column,
GEOCODE2,132
CLUSTER2,132
MSA,132
ADI,132
DMA,132
EIC13,0
OEDC5,0
OEDC4,0
OEDC3,0


In [74]:
# This part was made by the help of internet and long conversations with Chat GPT debugging the multiple errors that I got 🤣

    
# <span style="color:rgb(255, 0, 255)"> --- Apart for the values that were already sorted in previous labs and activities, the lab also asks to clean: ADI, DMA, and MSA. Use appropriate EDA technique where ever necessary.

In [75]:
describe_column(data, "ADI")

Unique Values:
[177.  13. 281.  67. 127. 185.  91. 251. 391. 181. 269.  83. 107. 249.
 291.  57. 227. 329. 351.  55. 133.  51.  73. 159. 201.  75. 233. 591.
 203. 175.  59. 105. 111. 173. 415. 323. 187.  65. 412. 109. 425. 405.
 197. 279. 645. 207.  89. 361. 209. 213.  15.  21. 113. 131. 469. 199.
 129. 441. 459. 235. 301.  39. 429. 285. 245. 335. 393. 577. 439. 455.
 383. 263.   0. 427. 307. 275. 119. 229. 273. 325. 371. 377. 315. 342.
 421. 219.  61.  93. 375. 283. 381. 243. 353. 613. 339. 367. 417. 299.
 271. 157.  77. 327.  53.  87. 451. 627.  71. 253. 363. 403. 115. 389.
  63. 321. 448. 237. 217. 241. 123. 221. 165. 277.  85. 319. 247.  69.
 379.  95.  17. 625. 462. 303. 179. 331. 231. 337. 257. 409. 457. 359.
 373.  nan 413. 215. 205. 313. 211. 305. 355. 419. 473. 423. 121. 433.
 431. 465. 445.  19. 467. 255. 437. 117. 449. 195. 435. 621. 295.  11.
 183. 265. 317.   9.   3. 151. 471. 293. 297. 603. 385.  33. 261.  25.
 443.  29. 357. 135.  35. 143. 153. 149.  47. 140.  43. 141. 2

In [76]:
#Let's see the most counted values: 

adi_value_counts = data["ADI"].value_counts()
adi_value_counts_df = pd.DataFrame(adi_value_counts).reset_index()
adi_value_counts_df.columns = ['ADI', 'Count']
adi_value_counts_df = adi_value_counts_df.sort_values('Count', ascending=False)
adi_value_counts_top_30 = adi_value_counts_df.head(30)

adi_value_counts_top_30


,ADI,Count
0,13.0,7296
1,51.0,4622
2,65.0,3765
3,57.0,2836
4,105.0,2617
5,197.0,2075
6,109.0,2054
7,131.0,1978
8,67.0,1847
9,201.0,1772


In [77]:
# I don't see anything to be cleaned in this column. 

In [78]:
describe_column(data, "DMA")

Unique Values:
[682. 803. 518. 862. 528. 691. 509. 643. 624. 659. 671. 527. 613. 716.
 770. 505. 717. 534. 560. 547. 571. 602. 800. 638. 618. 609. 820. 821.
 635. 675. 563. 819. 617. 637. 606. 709. 632. 807. 698. 623. 507. 627.
 524. 517. 539. 649. 868. 546. 529. 567. 825. 512. 669. 764. 575. 548.
 662. 811. 801. 652. 771. 603. 556. 622. 561. 724. 804. 633. 839. 686.
 650. 743. 619. 678. 753. 610. 745. 604. 749. 670. 765. 711. 553. 762.
 520. 503. 551. 515. 647. 544. 676. 752. 545. 754. 810. 790. 656. 755.
 641. 616. 628. 588. 581. 540. 658. 583. 866. 642. 746. 634. 648. 705.
 725. 744. 513. 612. 813. 531. 751. 500. 630. 611. 789. 582. 693. 692.
 828. 626. 710. 542. 855. 687. 679. 640. 722. 620. 881. 564. 522. 756.
 570. 718.  nan 530. 557. 625. 541. 657. 631. 550. 525. 605. 773. 519.
 535. 600. 734. 759. 757. 511. 798. 802. 644. 651. 702. 737. 767. 673.
 736. 636. 592. 758. 504. 639. 501. 506. 523. 760. 766. 646. 740. 574.
 598. 533. 661. 508. 537. 514. 510. 577. 549. 532. 521. 565. 5

In [79]:
#Let's see the most counted values: 

DMA_value_counts = data["DMA"].value_counts()
DMA_value_counts_df = pd.DataFrame(DMA_value_counts).reset_index()
DMA_value_counts_df.columns = ['DMA', 'Count']
DMA_value_counts_df = DMA_value_counts_df.sort_values('Count', ascending=False)
DMA_value_counts_top_30 = DMA_value_counts_df.head(30)

DMA_value_counts_top_30


,DMA,Count
0,803.0,7296
1,602.0,4632
2,807.0,3765
3,505.0,2839
4,819.0,2588
5,539.0,2222
6,524.0,2079
7,623.0,2009
8,862.0,1847
9,753.0,1780


In [80]:
# As earlier, I don't see how to clean this column. 

In [81]:
describe_column(data, "MSA")

Unique Values:
[   0. 4480. 9340. 5000. 2030. 3960. 5360. 3480.  760. 2160. 5960. 5120.
  440. 6580. 1600.  680. 3360. 7040. 2920. 3120. 6440.  640. 6120. 3000.
 7600. 2680. 5080. 1360. 2180. 4420. 1960. 6640. 7360. 5945. 1920. 1145.
 7520. 2800. 1520. 7510. 1620. 4520. 7320.  720. 4720. 1560. 8960.   40.
 6720.  520. 9360. 3710. 8280. 6760. 3160. 5560. 3600. 6780. 5800. 4120.
 5160. 5880.  380. 3440. 2710. 2020. 9040. 6200. 6880. 6920. 2760. 7080.
 4080. 2580. 4100. 5790. 4200.  600. 4680. 4040. 1640. 5775. 8720. 3980.
 2240. 1720. 5910. 9260.  200. 7400. 1020. 3290. 2720. 7240. 3800. 8120.
  870. 2320. 3760. 1040. 2840. 3880.  920. 6080.  460.  320. 1400. 3320.
 2640. 8200. 4890. 3660. 2670. 1000. 2520. 2440. 8520. 3720. 2080. 7500.
 3920. 1125. 4400.  840. 7485. 8750. 8735. 1150. 5280. 3285. 7460. 2620.
 7160. 4920. 8600.  880. 2985. 5920. 2655. 3560.   nan 3500. 7720. 7480.
  860. 2330. 7680. 5200. 3400. 3605. 2750. 8240. 3080. 5170. 3150.  480.
 6660. 3840. 2120. 3810. 2960. 7840.

In [82]:
#Let's see the most counted values: 

MSA_value_counts = data["MSA"].value_counts()
MSA_value_counts_df = pd.DataFrame(MSA_value_counts).reset_index()
MSA_value_counts_df.columns = ['MSA', 'Count']
MSA_value_counts_df = MSA_value_counts_df.sort_values('Count', ascending=False)
MSA_value_counts_top_30 = MSA_value_counts_df.head(30)

MSA_value_counts_top_30

,MSA,Count
0,0.0,21146
1,4480.0,4606
2,1600.0,4059
3,2160.0,2586
4,520.0,1685
5,6780.0,1525
6,5945.0,1500
7,7320.0,1472
8,7600.0,1457
9,3360.0,1414


In [ ]:
# Here I see a lot of 0 but It's fine, it's how the dataset works. 